In [2]:
import sys
sys.path.append('/Users/liangwang/GitHub/DataAnalysis2023')

In [3]:
import numpy as np
import os
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from scipy import signal 
from scipy import odr 
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from statsmodels.regression.rolling import RollingOLS

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [11]:
data_folder = Path(r'/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/CP-008 Session 25 - Pre-Pivotal2/experimental/prospective_8S22_CalFree_v14p_5day_AllRef')
# data_folder = Path(r'/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/Aggregate/eblinq19c_20_22/Experiment/prospective_AggData_CalFree_v14p_5day_neg3_pos3')
loaded_data = FileRead.load_pickle('alg_out.zip', data_folder, as_dict=True)
all_sensor_data = loaded_data['all_sensor_data']

In [13]:
loaded_data['data_dictionary']['br233210050167'].keys()

Index(['AcckT', 'acck', 'alsv_T', 'alsv', 'batt_T', 'batt', 'CEVT', 'CEV',
       'CGM_Time1', 'CGM_Sig1', 'CGM_Sig2', 'CGM_Sig3', 'CGM_Sig4', 'DexT',
       'DexG', 'freqT', 'freqHz', 'imag1_Time', 'imag1', 'imag2_Time', 'imag2',
       'imag3_Time', 'imag3', 'imag4_Time', 'imag4', 'real1_Time', 'real1',
       'real2_Time', 'real2', 'real3_Time', 'real3', 'real4_Time', 'real4',
       'LibreT', 'LibreV', 'TempT', 'TempV', 'YSIT', 'ysi'],
      dtype='object')

In [19]:
loaded_data['data_dictionary']['br233210050167']['CGM_Time1'][7902:7947]-loaded_data['data_dictionary']['br233210050167']['CGM_Time1'][0]

7902   2 days 17:10:07.600999832
7903   2 days 17:10:37.569000006
7904   2 days 17:11:07.520999909
7905   2 days 17:11:37.473000050
7906   2 days 17:12:07.440999985
7907   2 days 17:12:37.392999888
7908   2 days 17:13:07.360999823
7909   2 days 17:13:37.312999964
7910   2 days 17:14:07.280999899
7911   2 days 17:14:37.233000040
7912   2 days 17:15:07.200999975
7913   2 days 17:15:25.744999886
7914   2 days 17:15:55.713000059
7915   2 days 17:16:25.664999961
7916   2 days 17:16:55.632999897
7917   2 days 17:17:25.585000038
7918   2 days 17:17:55.552999973
7919   2 days 17:18:25.520999909
7920   2 days 17:18:55.488999844
7921   2 days 17:19:25.440999985
7922   2 days 17:19:55.408999920
7923   2 days 17:20:25.376999855
7924   2 days 17:20:55.345000029
7925   2 days 17:21:25.312999964
7926   2 days 17:21:55.280999899
7927   2 days 17:22:25.248999834
7928   2 days 17:22:55.217000008
7929   2 days 17:23:25.200999975
7930   2 days 17:23:55.168999910
7931   2 days 17:24:25.136999846
7932   2 d

In [15]:
loaded_data['data_dictionary']['br233210050167']['CGM_Sig4'][7902:7947]

7902    0.171
7903    0.080
7904    0.032
7905    0.024
7906    0.021
7907    0.032
7908    0.049
7909    0.082
7910    0.093
7911    0.131
7912    0.600
7913    0.173
7914    0.168
7915    0.164
7916    0.164
7917    0.137
7918    0.043
7919    0.015
7920    0.011
7921    0.010
7922    0.012
7923    0.347
7924    0.170
7925    0.155
7926    0.151
7927    0.157
7928    0.158
7929    0.157
7930    0.157
7931    0.149
7932    0.155
7933    0.154
7934    0.145
7935    0.064
7936    0.017
7937    0.014
7938    0.009
7939    0.008
7940    0.011
7941    0.009
7942    0.008
7943    0.008
7944    0.009
7945    0.008
7946    0.365
Name: CGM_Sig4, dtype: float64

In [3]:
ch_str = ['1', '3', '4']
gluc_col_prefix = 'model_output_s'
gluc_col_suffix = ''
time_to_match = 'UTC_Time'
def cgm_acck_linear(B, x):
    return B[0]+B[1]*x
orthogonal_model = odr.Model(cgm_acck_linear)
use_acck_in_case = False

In [4]:
biolinq_col_include = ['Time','time_from_start','sensor_id','UTC_Time','Temperature','biolinq']
biolinq_col_include_ch1 = ['cur1', 'deg1_103', 'mag1_103','deg1_1007', 'mag1_1007', 'deg1_10000', 'mag1_10000', 'model_output_s1', 'biolinq_v14p_s1']
biolinq_col_include_ch3 = ['cur3', 'deg3_103', 'mag3_103','deg3_1007', 'mag3_1007', 'deg3_10000', 'mag3_10000', 'model_output_s3', 'biolinq_v14p_s3']
biolinq_col_include_ch4 = ['cur4', 'deg4_103', 'mag4_103','deg4_1007', 'mag4_1007', 'deg4_10000', 'mag4_10000', 'model_output_s4', 'biolinq_v14p_s4']

ref_value_col_name = 'Ref_value'
ref_type_col_name = 'Ref_type'
ref_value_orig_col_name = 'Ref_value_orig'

In [5]:
aggregated_df = pd.DataFrame()
for sensor in all_sensor_data.keys():
# for sensor in ['31360030051', '31360030069', '31360030071', '31360020004', '31360020006', '31360020023', '31360020026', '31360020032', '31360020034', '31360020038', '31360020041']:
# for sensor in ['0br231790020085']:    
    # print(sensor)
    current_sensor_data = all_sensor_data[sensor].copy()
    biolinq_exist = False
    acck_exist = False
    ysi_exist = False
    Libre_exist = False
    Dexcom_exist = False
    ref_for_match_exist = False
    if 'biolinq' in current_sensor_data.keys():
        biolinq_exist = True
        biolinq_df = current_sensor_data['biolinq'].copy()
    if 'acck' in current_sensor_data.keys():
        acck_exist = True
        acck_df = current_sensor_data['acck'].copy()
    if 'ysi' in current_sensor_data.keys():
        ysi_exist = True
        ysi_df = current_sensor_data['ysi'].copy()
    if 'Libre' in current_sensor_data.keys():
        Libre_exist = True
        Libre_df = current_sensor_data['Libre'].copy()
    if 'Dexcom' in current_sensor_data.keys():
        Dexcom_exist = True
        Dexcom_df = current_sensor_data['Dexcom'].copy()

    if 'biolinq' not in biolinq_df.columns:
        continue
    # biolinq_df = biolinq_df.drop(columns=biolinq_col_remove)

    """ Calibrate Libre and Dexcom, determine the reference """
    intercept_Libre = np.nan
    slope_Libre = np.nan
    intercept_Dexcom = np.nan
    slope_Dexcom = np.nan
    ref_df = pd.DataFrame()
    if acck_exist and not acck_df.empty:
        if Libre_exist and not Libre_df.empty:
            Libre_acck_df = pd.merge_asof(left=acck_df.dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=Libre_df.dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                       on=time_to_match, direction='nearest', tolerance=pd.Timedelta('7.5min'))
            Libre_acck_df = Libre_acck_df.dropna(subset=['acck'])
            Libre_acck_df = Libre_acck_df.dropna(subset=['Libre'])
            # Orthogonal regression
            current_data = odr.Data(Libre_acck_df['acck'],Libre_acck_df['Libre'])
            orthogonal_reg = odr.ODR(current_data,orthogonal_model,beta0=[0.0,1.0])
            orthogonal_reg_outcome = orthogonal_reg.run()
            intercept_Libre_tmp = orthogonal_reg_outcome.beta[0]
            slope_Libre_tmp = orthogonal_reg_outcome.beta[1]
            if (Libre_acck_df.shape[0]>=10) and (Libre_acck_df['acck'].max()-Libre_acck_df['acck'].min()>=30) and (intercept_Libre_tmp>=-60) and (intercept_Libre_tmp<=60) and (slope_Libre_tmp>=0.4) and (slope_Libre_tmp<=1.6):
                intercept_Libre = intercept_Libre_tmp
                slope_Libre = slope_Libre_tmp
                ref_df = Libre_df
                ref_df.rename(columns={'Libre':ref_value_orig_col_name},inplace=True)
                ref_df[ref_value_col_name]=ref_df[ref_value_orig_col_name].subtract(intercept_Libre).divide(slope_Libre)
                ref_df[ref_type_col_name] = 'Libre'
                ref_for_match_exist = True
        elif Dexcom_exist and not Dexcom_df.empty:
            Dexcom_acck_df = pd.merge_asof(left=acck_df.dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=Dexcom_df.dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                       on=time_to_match, direction='nearest', tolerance=pd.Timedelta('7.5min'))
            Dexcom_acck_df = Dexcom_acck_df.dropna(subset=['acck'])
            Dexcom_acck_df = Dexcom_acck_df.dropna(subset=['Dexcom'])
            # Orthogonal regression
            current_data = odr.Data(Dexcom_acck_df['acck'],Dexcom_acck_df['Dexcom'])
            orthogonal_reg = odr.ODR(current_data,orthogonal_model,beta0=[0.0,1.0])
            orthogonal_reg_outcome = orthogonal_reg.run()
            intercept_Dexcom_tmp = orthogonal_reg_outcome.beta[0]
            slope_Dexcom_tmp = orthogonal_reg_outcome.beta[1]
            if (Dexcom_acck_df.shape[0]>=10) and (Dexcom_acck_df['acck'].max()-Dexcom_acck_df['acck'].min()>=30) and (intercept_Dexcom_tmp>=-60) and (intercept_Dexcom_tmp<=60) and (slope_Dexcom_tmp>=0.4) and (slope_Dexcom_tmp<=1.6):
                intercept_Dexcom = intercept_Dexcom_tmp
                slope_Dexcom = slope_Dexcom_tmp
                ref_df = Dexcom_df
                ref_df.rename(columns={'Dexcom':ref_value_orig_col_name},inplace=True)
                ref_df[ref_value_col_name]=ref_df[ref_value_orig_col_name].subtract(intercept_Dexcom).divide(slope_Dexcom)                
                ref_df[ref_type_col_name] = 'Dexcom'
                ref_for_match_exist = True
        elif use_acck_in_case:
            ref_df = acck_df
            ref_df.rename(columns={'acck':ref_value_orig_col_name},inplace=True)
            ref_df[ref_value_col_name]=ref_df[ref_value_orig_col_name]           
            ref_df[ref_type_col_name] = 'Acck'
            ref_for_match_exist = True

    biolinq_ref_df = pd.DataFrame()
    if biolinq_exist and not biolinq_df.empty and not ref_df.empty:
        biolinq_df['cur1_below_20pA'] = biolinq_df['cur1'].le(0.02)
        biolinq_df['cur3_below_20pA'] = biolinq_df['cur3'].le(0.02)
        biolinq_df['cur4_below_20pA'] = biolinq_df['cur4'].le(0.02)
        biolinq_df['ratio_future_cur1_below_20pA'] = biolinq_df[::-1]['cur1_below_20pA'].expanding().mean()[::-1]
        biolinq_df['ratio_future_cur3_below_20pA'] = biolinq_df[::-1]['cur3_below_20pA'].expanding().mean()[::-1]
        biolinq_df['ratio_future_cur4_below_20pA'] = biolinq_df[::-1]['cur4_below_20pA'].expanding().mean()[::-1]
        index_valid = biolinq_df[(biolinq_df['ratio_future_cur1_below_20pA'].le(0.95)) | (biolinq_df['ratio_future_cur3_below_20pA'].le(0.95)) | (biolinq_df['ratio_future_cur4_below_20pA'].le(0.95))].index
        biolinq_df = biolinq_df.loc[index_valid,:]
        biolinq_cur1_df = biolinq_df[biolinq_col_include+biolinq_col_include_ch1+['ratio_future_cur1_below_20pA']]
        biolinq_cur1_df.rename(columns={'Time':'Time_biolinq',
                                        'cur1':'current',
                                        'model_output_s1':'model_output',
                                        'biolinq_v14p_s1':'valid_model_output',
                                        'deg1_103':'deg_103',
                                        'mag1_103':'mag_103',
                                        'deg1_1007':'deg_1007',
                                        'mag1_1007':'mag_1007',
                                        'deg1_10000':'deg_10000',
                                        'mag1_10000':'mag_10000',
                                        'ratio_future_cur1_below_20pA':'ratio_future_current_below_20pA'
                                        },inplace=True)
        biolinq_cur1_df.insert(biolinq_cur1_df.columns.get_loc('sensor_id')+1,'Channel','Ch1')

        biolinq_cur3_df = biolinq_df[biolinq_col_include+biolinq_col_include_ch3+['ratio_future_cur3_below_20pA']]
        biolinq_cur3_df.rename(columns={'Time':'Time_biolinq',
                                        'cur3':'current',
                                        'model_output_s3':'model_output',
                                        'biolinq_v14p_s3':'valid_model_output',
                                        'mag3_103':'mag_103',
                                        'deg3_103':'deg_103',
                                        'deg3_1007':'deg_1007',
                                        'mag3_1007':'mag_1007',
                                        'deg3_10000':'deg_10000',
                                        'mag3_10000':'mag_10000',
                                        'ratio_future_cur3_below_20pA':'ratio_future_current_below_20pA'
                                        },inplace=True)        
        biolinq_cur3_df.insert(biolinq_cur3_df.columns.get_loc('sensor_id')+1,'Channel','Ch3')        

        biolinq_cur4_df = biolinq_df[biolinq_col_include+biolinq_col_include_ch4+['ratio_future_cur4_below_20pA']]
        biolinq_cur4_df.rename(columns={'Time':'Time_biolinq',
                                        'cur4':'current',
                                        'model_output_s4':'model_output',
                                        'biolinq_v14p_s4':'valid_model_output',
                                        'mag4_103':'mag_103',
                                        'deg4_103':'deg_103',
                                        'deg4_1007':'deg_1007',
                                        'mag4_1007':'mag_1007',
                                        'deg4_10000':'deg_10000',
                                        'mag4_10000':'mag_10000',
                                        'ratio_future_cur4_below_20pA':'ratio_future_current_below_20pA'
                                        },inplace=True)  
        biolinq_cur4_df.insert(biolinq_cur4_df.columns.get_loc('sensor_id')+1,'Channel','Ch4')          
        biolinq_df_stack = pd.concat([biolinq_cur1_df, biolinq_cur3_df, biolinq_cur4_df],ignore_index=True)
        biolinq_ref_df = pd.merge_asof(left=biolinq_df_stack.dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=ref_df.dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                    on=time_to_match, direction='nearest', tolerance=pd.Timedelta('5min'))
        biolinq_ref_df = biolinq_ref_df.dropna(subset=[ref_value_col_name])
        biolinq_ref_df = biolinq_ref_df.dropna(subset=['model_output'])   
        aggregated_df = pd.concat([aggregated_df,biolinq_ref_df],ignore_index=True)

In [6]:
aggregated_df.to_csv('/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Documents - Clinical Data Analysis/Data/Aggregate/eblinq19c_20_22/Experiment/prospective_AggData_CalFree_v14p_5day_neg3_pos3/performance/biolinq vs adj Libre Dexcom.csv',header=True)